# Exercise for OR-INF transport module 2021
Introduction to the basics of transport modelling. If you want to learn more by yourself, have a read in [Modelling Transport by Ortúzar and Willumsen](https://www.wiley.com/en-us/Modelling+Transport%2C+4th+Edition-p-9780470760390) or have a look into a more complex [transport model for Germany](https://github.com/marlinarnz/quetzal_germany).

In [ ]:
# Libraries we need
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import geometry
from geopy.distance import geodesic

The `pandas` and `numpy` libraries are core tools for all kinds of data handling and analysis in Python. `pandas` allows easy and quick handling of data in so-called DataFrames ([pandas tutorials](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/index.html)). `geopandas` is an extension of `pandas` which adds functionalities for geometric operations. `numpy` is the essential library for mathematical operations. `shapely.geometry` provides classes for geometric operations, while `geopy.distance.geodesic` provides a method for calculation geometric distances in meters.

## Create a test network
Before we get startet we need nodes and links to carry out transport analysis.

In [ ]:
# Create our test network nodes
nodes = pd.DataFrame(
    data={'name': ['Berlin', 'Hamburg', 'Frankfurt', 'Stuttgart'],
          'lat': [52.518611, 53.550556, 50.110556, 48.775556],
          'lon': [13.408333, 9.993333, 8.682222, 9.182778]})
nodes

In [ ]:
# Create a GeoDataFrame. It looks similar to the normal
# DataFrame but has additional attributes like plotting
nodes = gpd.GeoDataFrame(
    nodes, geometry=gpd.points_from_xy(nodes.lon, nodes.lat))

In [ ]:
# Load basemap shapes
countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ger = countries[countries['name']=='Germany']
# Plot test network nodes
nodes.plot(c='r', ax=ger.plot())

In [ ]:
# Create links from every node to every node for each mode
links = pd.DataFrame(
    {'a': [0,0,0,1,1,1,2,2,2,3,3,3]*2,
     'b': [1,2,3,0,2,3,0,1,3,0,1,2]*2,
     'mode': ['rail']*12+['car']*12})
# Add the geometry attributes
links = gpd.GeoDataFrame(
    links, geometry=[geometry.LineString([
        nodes.loc[row.a, 'geometry'],nodes.loc[row.b, 'geometry']
    ]) for _, row in links.iterrows()])
# Add the distance in km
links['dist'] = [geodesic(l.coords[0], l.coords[-1]).km
                 for l in list(links['geometry'])]
# Add level of service (LoS) attribute time in h
links['time'] = links['dist'] / 110 #km/h is about average
# Adjust time of rail services
links.loc[links['mode']=='rail', 'time'] = [
    1.8,4.2,6.2, 1.8,3.5,5.5, 4.2,3.5,1.5, 6.2,5.5,1.5]
# Sort values by OD-pairs
links = links.sort_values(['a', 'b']).reset_index(drop=True)

In [ ]:
# In our links table, every two rows are one OD pair now
links

In [ ]:
# Plot
nodes.plot(c='r', ax=links.plot(color='y', ax=ger.plot()))

## Step 1: Calculate mode choice probabilities
Now we can calculate the probabilities of using one or the other mode in a logit formulation. First, we use the level of service (LoS) attribute time.

In [ ]:
# Mathematical formulation of the MNL model
def mnl(od_pair, los_col_names=['time'], beta=[1.5]):
    '''Calculates the probabilities for each mode on the given
        OD pair. One OD pair are all links between the same
        origin and destination (all modes of analysis)
    @param od_pair: pandas DataFrame slice of links from one 
        origin to one destination for all modes of analysis
    @param los_col_names: list of DataFrame column names for all
        LoS attributes of analysis (in the same order as betas)
    @param beta: list of numerical calibration factors for 
        every LoS attribute given
    @return: list of probabilities for each mode in od_pair
        in order of appearence'''
    v = [- sum([row[los_col_names[i]] * beta[i]
                for i in range(len(los_col_names))]
              ) for _, row in od_pair.iterrows()]
    p = [np.exp(val) / sum([np.exp(m) for m in v])
         for val in v]
    return p

Use the above function to calculate a new column `probs_1` in links. Hint: Iterate over OD-pairs, give them to the MNL function and save the returned values to the new column of this OD pair

With `.loc[]` you can access rows, columns and single values in tables based on their index and column names ([docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html#pandas.DataFrame.loc)).
With `.iloc[]` you can access rows, columns and single values in tables based on their index and column numbers ([docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html#pandas.DataFrame.iloc)).

In [ ]:
# There are many ways to get probabilities as a new column
# This might be easily understandable
links['probs_1'] = 

In [ ]:
links

In [ ]:
# Calculate the mean of the results
links.groupby('mode').agg({'probs_1': 'mean'})

## Step 2: New LoS attribute travel cost
After adding a new column for travel cost, you can apply the above formula again. First, add a column `cost` to links. Cost in EUR have the following dependencies with trip distance:

- car (average direct cost in Germany without vehicle maintenance or purchase cost): 0.12 * distance
- rail (ticket prices without Bahncard or Sparpreis): 0.7 * distance^0.8

Hint: Use `np.power(base, exponent)` for calculation of rail fares

In [ ]:
links['cost'] = 

In [ ]:
links['probs_2'] = 

In [ ]:
links

In [ ]:
# Calculate the mean of all results
links.groupby('mode').agg({'probs_1': 'mean', 'probs_2': 'mean'})

## Step 3: Political action for sustainable transport
Now assume: Due to tax reductions for rail operators rail services cost 10% less. Moreover, a speed limit of 120km/h on motorways is enforced, which reduces average car travel speed to 100km/h. What are the new mode choice probabilities?

In [ ]:
links['probs_3'] = 

In [ ]:
links

In [ ]:
# Calculate the mean of all results
links.groupby('mode').agg({'probs_1': 'mean',
                           'probs_2': 'mean',
                           'probs_3': 'mean'})

## Step 4: Trip volumes
Given an OD-matrix, you can calculate trip columes on each link. How many people are travelling where using which mode?

What is the summed up travel time?

In [ ]:
# Number of trips per year
od_matrix = pd.read_csv('od_matrix.csv', header=None)
od_matrix.columns = ['a', 'b', 'volumes']
od_matrix.set_index(['a', 'b'], inplace=True)

In [ ]:
od_matrix

In [ ]:
# Use a dictionary to translate numbers in columns 
# a and b of links table to names in the OD matrix
name_dict = nodes['name'].to_dict()
# Access items as follows
name_dict[0]

In [ ]:
# Calculate trip volumes
links['volumes'] = 

In [ ]:
links.head(4)

In [ ]:
# Calculate passenger kilometer (pkm)
# You can use all mathematical operations on whole columns
links['pkm'] = 

In [ ]:
links.groupby('mode').agg({'pkm': 'mean'})

In [ ]:
# Plot the volumes on the network as width of links
# for road transport
nodes.plot(c='r', ax=links.loc[links['mode']=='car'].plot(
    color='y', linewidth=[v / links['volumes'].mean()
                          for v in links['volumes']], ax=ger.plot()))

In [ ]:
# Plot the volumes on the network as width of links
# for rail transport
nodes.plot(c='r', ax=links.loc[links['mode']=='rail'].plot(
    color='y', linewidth=[v / links['volumes'].mean()
                          for v in links['volumes']], ax=ger.plot()))

In [ ]:
# Aggregated travel time
t = 
t.sum()

## Step 5: Capacities and congestion
So far, travel on our test network was not restricted. However, in reality roads become congested and trains crowded. Given the capacities of links and a volume-dependent congestion formula, how do people travel now? What is the new aggregated travel time?

If you want, you can try multiple iterations to reach an equilibrium between volumes and capacities.

In [ ]:
# Average occupancy in ICE trains is low in Germany while
# main motorways are quite packed
links['cap'] = [od_matrix.loc[
    (name_dict[row['a']], name_dict[row['b']]), 'volumes'] / 2
#                if row['mode']=='car' else row['volumes']*3
                for _, row in links.iterrows()]

In [ ]:
def travel_time(volumes, capacity, time):
    '''Calculates the travel time of a link with capacity
    restriction applying the formula used by Smock (1962)
    @param volumes: volume of trips on this link
    @param capacity: capacity for trips on this link
    @param time: travel time without capacity restriction
    @return: new travel time'''
    return 0.5 * time * np.exp(volumes / capacity)

In [ ]:
links['time_2'] = [travel_time(row['volumes'], row['cap'], row['time'])
                   if row['mode']=='car' else row['time']
                   for _, row in links.iterrows()]

In [ ]:
links.head(2)

In [ ]:
links['volumes_2'] = np.nan
links['probs_4'] = np.nan

In [ ]:
for j in range(10):
    # Probabilities
    
    # New volumes
    links['volumes_2'] = 
    # New travel times
    links['time_2'] = 
    print('Iteration {} av. mode shares: car {}; rail {}'.format(
        j, links.loc[links['mode']=='car', 'probs_4'].mean(),
        links.loc[links['mode']=='rail', 'probs_4'].mean()))

In [ ]:
links.sample(4)

In [ ]:
# Calculate the mean of all results
links.groupby('mode').agg({'probs_1': 'mean',
                           'probs_2': 'mean',
                           'probs_3': 'mean',
                           'probs_4': 'mean'})

In [ ]:
# Aggregated travel time
# Compare to step 5
t = 
t.sum()

## Step 6: Impact of lorries
Looking into the future, roads will become even more congested by lorries due to the rising demand for goods transport, if there is no political ambition for a shift to rail in freight transport. Given these new capacities, how does the mode share in passenger transport change? And the aggregated travel time?

In [ ]:
links['cap_2'] = [row['cap']*0.9
                  if row['mode']=='car' else row['cap']
                  for _, row in links.iterrows()]

# End
Congrats! You've learned the basics of open source transport modelling with Python. If you want you can go on with altering capacities, adding new nodes or modes or even adding a new LoS attribute.